In [ ]:
!pip install transformers datasets tensorflow sentence-transformers faiss-cpu flask

In [ ]:

question,answer
What is a fixed deposit?,A fixed deposit is a savings instrument with a fixed interest rate and period.
How do I apply for a credit card?,You can apply online via the bank website or visit a branch with required documents.
Difference between savings and current account?,Savings accounts earn interest and have withdrawal limits; current accounts are for business with high transactions.

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd
import numpy as np

# Load dataset
data = pd.read_csv("financial_qa.csv")
questions = data['question'].tolist()
answers = data['answer'].tolist()

# Load T5 model and tokenizer
model_name = "flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

# LoRA Adapter: trainable low-rank layers
class LoRALayer(tf.keras.layers.Layer):
    def __init__(self, original_layer, rank=4, **kwargs):
        super().__init__(**kwargs)
        self.original_layer = original_layer
        self.original_layer.trainable = False
        self.A = tf.keras.layers.Dense(rank, use_bias=False)
        self.B = tf.keras.layers.Dense(original_layer.units, use_bias=False)

    def call(self, inputs):
        original_out = self.original_layer(inputs)
        lora_out = self.B(self.A(inputs))
        return original_out + lora_out

# Example: attach LoRA to decoder dense layers
for layer in model.decoder._layers:
    if isinstance(layer, tf.keras.layers.Dense):
        layer.trainable = False  # freeze original
        lora = LoRALayer(layer)
        layer.call = lora.call

# Tokenize dataset
def tokenize(q, a):
    inputs = tokenizer(q, max_length=128, truncation=True, padding='max_length', return_tensors='tf')
    targets = tokenizer(a, max_length=128, truncation=True, padding='max_length', return_tensors='tf')
    inputs['labels'] = targets['input_ids']
    return inputs

input_ids_list, attention_mask_list, labels_list = [], [], []
for q, a in zip(questions, answers):
    tokenized = tokenize(q, a)
    input_ids_list.append(tokenized['input_ids'][0])
    attention_mask_list.append(tokenized['attention_mask'][0])
    labels_list.append(tokenized['labels'][0])

train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": tf.stack(input_ids_list), "attention_mask": tf.stack(attention_mask_list)},
    tf.stack(labels_list)
)).batch(8)

# Compile and train
model.compile(optimizer='adam', loss=model.hf_compute_loss)
model.fit(train_dataset, epochs=1)

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss

# Sample financial documents
documents = [
    "A fixed deposit is a savings account with fixed interest.",
    "Credit cards can be applied online or offline with KYC documents.",
    "Savings accounts earn interest; current accounts are for business accounts."
]

# Create embeddings
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = embed_model.encode(documents)

# Build FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Optional: sentiment (can ignore if simple)
from transformers import pipeline
sentiment_pipe = pipeline("sentiment-analysis")

def retrieve_docs(question, top_k=1):
    q_embed = embed_model.encode([question])
    distances, indices = index.search(q_embed, top_k)
    return " ".join([documents[i] for i in indices[0]])

@app.route('/chat', methods=['POST'])
def chat():
    user_question = request.json.get('message')

    # Retrieve relevant context
    context = retrieve_docs(user_question)

    # Build prompt
    prompt = f"Context: {context} Question: {user_question}"

    # Tokenize and generate
    inputs = tokenizer(prompt, max_length=256, truncation=True, return_tensors='tf')
    outputs = model.generate(inputs['input_ids'], max_length=128)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return jsonify({"response": answer})

@app.route('/greet', methods=['GET'])
def greet():
    return jsonify({"message": "Hello! I am your Smart Financial Advisor. Ask me anything about finance."})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)